In [1]:
#! pip install -U "datasets<4.0.0" torch
!pip install ipython-autotime tqdm_joblib --quiet
%load_ext autotime
! pwd


/kaggle/working
time: 113 ms (started: 2025-08-12 01:23:00 +00:00)


In [2]:
! ls -al /kaggle/input/amazon-processed

DATA_PATH = "/kaggle/input/amazon-processed"
FINISHED_EPOCHS = 6  # 0-indexed

total 1256360
drwxr-xr-x 2 nobody nogroup         0 Aug 12 01:08 .
drwxr-xr-x 4 root   root         4096 Aug 12 01:22 ..
-rw-r--r-- 1 nobody nogroup     49572 Aug 12 01:08 bag_of_words_1754014600.591909.json
-rw-r--r-- 1 nobody nogroup   1531208 Aug 12 01:08 model_epoch2_1754046175.4568558.pth
-rw-r--r-- 1 nobody nogroup   1531144 Aug 12 01:08 model_epoch6_1754644752.4333465.pth
-rw-r--r-- 1 nobody nogroup 641693788 Aug 12 01:08 X_processed_1754085057.625196.pt
-rw-r--r-- 1 nobody nogroup 641693984 Aug 12 01:08 Xt_processed_IndustrialScientific_1754088738.538544.pt
time: 127 ms (started: 2025-08-12 01:23:00 +00:00)


In [3]:
from datasets import load_dataset

from joblib import Parallel, delayed
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm
from tqdm_joblib import tqdm_joblib
# from matplotlib import pyplot as plt

import json
import os
import time

mps = torch.device("mps")

dataset = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023", "raw_review_Software", trust_remote_code=True
)
# TODO: download to volume
"""
format:
{rating: int, title: str, text: str}
"""

README.md: 0.00B [00:00, ?B/s]

Amazon-Reviews-2023.py: 0.00B [00:00, ?B/s]

Software.jsonl:   0%|          | 0.00/1.87G [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

'\nformat:\n{rating: int, title: str, text: str}\n'

time: 1min 32s (started: 2025-08-12 01:23:00 +00:00)


In [4]:
print(f"MPS available: {torch.backends.mps.is_available()}")
cuda_available = torch.cuda.is_available()
DEVICE = torch.device("cuda" if cuda_available else "cpu")
print(cuda_available)

MPS available: False
True
time: 163 ms (started: 2025-08-12 01:24:33 +00:00)


In [5]:
n_samples = len(dataset["full"]["text"])  # 480000
n_samples

# linear to size of dataset
# linear to number of params
# linear to number of batches (not size)

4880181

time: 5.76 s (started: 2025-08-12 01:24:33 +00:00)


In [6]:
# n_samples = n_samples // 100
X = dataset["full"]["text"][:n_samples]
# X_test = dataset["full"]["text"][200000:225000]

y = torch.tensor(dataset["full"]["rating"][:n_samples]) >= 4

y

tensor([False,  True,  True,  ...,  True,  True,  True])

time: 8.71 s (started: 2025-08-12 01:24:39 +00:00)


In [7]:
DO_VALIDATION = True

if DO_VALIDATION:
    test_dataset = load_dataset(
        "McAuley-Lab/Amazon-Reviews-2023", "raw_review_Industrial_and_Scientific", trust_remote_code=True
    )

    # test_samples = 480000
    test_samples = len(test_dataset["full"]["text"])
    print(test_samples)

    
    Xt = dataset["full"]["text"][:test_samples]
    yt = torch.tensor(dataset["full"]["rating"][:test_samples]) >= 4

Industrial_and_Scientific.jsonl:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

5183005
time: 1min 53s (started: 2025-08-12 01:24:47 +00:00)


In [8]:
bag_of_words = {}

time: 339 µs (started: 2025-08-12 01:26:41 +00:00)


In [9]:
import re


def split_text(text: str) -> list[str]:
    text = text.lower().strip()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    words = re.split(r"\s+", text)
    return words


total_words = 0


def add_to_bag(text: str) -> None:
    global total_words
    words = split_text(text)
    total_words += len(words)
    for word in words:
        bag_of_words[word] = bag_of_words.get(word, 0) + 1


time: 753 µs (started: 2025-08-12 01:26:41 +00:00)


In [10]:
# fmt: off
ignored_words = [
    # articles
    "a", "an", "the",
    # pronouns
    "i", "you", "he", "she", "it", "we", "they", "me", "him", "her", "us", "them", "my", "your", "his", "her", "its", "our", "their","his","her","its","our","their",
    # prepositions
    "at", "in", "on", "by", "for", "with", "without", "to", "from", "of", "about", "under", "over", "through", "between", "among", "during", "before", "after", "above", "below", "up", "down", "out", "off", "into", "onto", "upon", "within", "across", "along", "around", "behind", "beside", "beyond", "inside", "outside", "toward", "towards", "underneath", "against", "beneath", "near", "next", "past", "since", "until", "via",
    # conjunctions
    "and", "or", "but", "so", "yet", "nor", "as", "if", "when", "while", "because", "although", "though", "unless", "whereas", "however", "therefore", "moreover", "furthermore", "nevertheless", "meanwhile",
    # common verbs
    "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "do", "does", "did", "will", "would", "could", "should", "can", "may", "might", "must", "shall", "get", "got", "go", "went", "come", "came", "see", "saw", "know", "knew", "think", "thought", "say", "said", "tell", "told", "make", "made", "take", "took", "give", "gave", "find", "found", "use", "used", "work", "worked", "look", "looked", "seem", "seemed", "feel", "felt", "try", "tried", "leave", "left", "put", "set", "keep", "kept", "let", "run", "ran", "move", "moved", "live", "lived", "bring", "brought", "happen", "happened", "write", "wrote", "sit", "sat", "stand", "stood", "lose", "lost", "pay", "paid", "meet", "met", "include", "included", "continue", "continued", "turn", "turned", "follow", "followed", "want", "wanted", "need", "needed", "like", "liked", "help", "helped", "talk", "talked", "become", "became", "show", "showed", "hear", "heard", "play", "played", "run", "ran", "move", "moved", "live", "lived", "believe", "believed", "hold", "held", "bring", "brought", "happen", "happened", "write", "wrote", "provide", "provided", "sit", "sat", "stand", "stood", "lose", "lost", "pay", "paid", "meet", "met", "include", "included",
    # adverbs
    "very", "really", "quite", "just", "only", "also", "too", "so", "more", "most", "much", "many", "well", "good", "better", "best", "bad", "worse", "worst", "little", "less", "least", "big", "bigger", "biggest", "small", "smaller", "smallest", "long", "longer", "longest", "short", "shorter", "shortest", "high", "higher", "highest", "low", "lower", "lowest", "first", "last", "next", "previous", "new", "old", "young", "great", "right", "wrong", "true", "false", "sure", "probably", "maybe", "perhaps", "definitely", "certainly", "absolutely", "completely", "totally", "exactly", "almost", "nearly", "hardly", "barely", "quite", "rather", "pretty", "fairly", "somewhat", "slightly", "extremely", "incredibly", "amazingly", "surprisingly", "unfortunately", "fortunately", "obviously", "clearly", "apparently", "generally", "usually", "normally", "typically", "often", "sometimes", "rarely", "never", "always", "already", "still", "yet", "soon", "now", "then", "here", "there", "where", "everywhere", "anywhere", "somewhere", "nowhere", "how", "why", "what", "when", "who", "which", "whose", "whom",
    # numbers and quantities
    "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten", "first", "second", "third", "another", "other", "others", "some", "any", "all", "each", "every", "both", "either", "neither", "none", "few", "several", "many", "most", "much", "little", "less", "more", "enough", "plenty",
    # misc common words
    "thing", "things", "something", "anything", "nothing", "everything", "someone", "anyone", "everyone", "no", "yes", "ok", "okay", "please", "thanks", "thank", "welcome", "hello", "hi", "bye", "goodbye", "sorry", "excuse", "pardon", "way", "ways", "time", "times", "day", "days", "year", "years", "place", "places", "people", "person", "man", "woman", "child", "children", "life", "world", "home", "house", "work", "job", "money", "business", "company", "part", "parts", "number", "numbers", "group", "groups", "problem", "problems", "question", "questions", "answer", "answers", "fact", "facts", "example", "examples", "case", "cases", "point", "points", "idea", "ideas", "information", "data", "result", "results", "change", "changes", "end", "beginning", "start", "finish", "side", "sides", "hand", "hands", "eye", "eyes", "head", "face", "back", "front", "top", "bottom", "left", "right", "inside", "outside", "important", "different", "same", "such", "even", "still", "however", "though", "although", "since", "while", "during", "before", "after", "until", "unless", "because", "if", "whether", "that", "this", "these", "those", "there", "here", "where", "when", "how", "why", "what", "who", "which", "whose", "whom",
    # common contractions
    "ive", "im",
    # parsing-specific
    "", "br"
]
# fmt: on

bag_of_words = {}

if DO_VALIDATION:
    with open(f'/kaggle/input/amazon-processed/bag_of_words_1754014600.591909.json', 'r') as file:
        bag_of_words = json.load(file)
        print("loaded bag from file")

else:
    for observation in X:
        add_to_bag(observation)
    
    print(len(bag_of_words))
    
    
    bag_of_words = {
        k: v for k, v in bag_of_words.items() if v > 10 and k not in ignored_words
    }
    print(len(bag_of_words))
    bag_size = 3000
    bag_of_words = dict(
        sorted(bag_of_words.items(), key=lambda x: x[1], reverse=True)[:bag_size]
    )
    
    print(len(bag_of_words))
    print("avg words per review:", total_words / len(X))

    BOW_PATH = f"/kaggle/working/bag_of_words_{time.time()}.json"
    json.dump(bag_of_words, open(BOW_PATH, "w"))
    print("Saved BoW to", BOW_PATH)


loaded bag from file
time: 14.7 ms (started: 2025-08-12 01:26:41 +00:00)


In [11]:
def vectorize(text: str, word_to_idx: dict[str, int]) -> torch.Tensor:
    words = split_text(text)

    word_counts = {}  # don't allocate all indices

    for word in words:
        idx = word_to_idx.get(word)
        if idx is not None:
            word_counts[idx] = word_counts.get(idx, 0) + 1

    if not word_counts:
        return torch.sparse_coo_tensor(
            torch.empty(
                (1, 0),  # 1 dimension, 0 non-zero elements (nnz)
                dtype=torch.long
            ),
            [],
            (len(word_to_idx),),
            dtype=torch.float16,
        )

    indices = torch.tensor([list(word_counts.keys())], dtype=torch.long)
    values = torch.tensor(list(word_counts.values()), dtype=torch.float16)

    return torch.sparse_coo_tensor(
        indices, values, (len(word_to_idx),), dtype=torch.float16
    )


def vectorize_parallel(
    texts: list[str], word_to_idx: dict[str, int], n_jobs: int = -1
) -> torch.Tensor:
    with tqdm_joblib(tqdm(total=len(texts))):
        # 15s vs. threading: 11.8s on 1% of dataset
        vectors = Parallel(n_jobs=n_jobs, backend="threading")(
            delayed(vectorize)(text, word_to_idx) for text in texts
        )
    return torch.stack(vectors)#.to(DEVICE)


def preprocess(X: list[str]) -> torch.Tensor:
    word_to_idx = {word: idx for idx, word in enumerate(bag_of_words.keys())}

    return vectorize_parallel(X, word_to_idx)

SAVED_VECTORIZED = "/kaggle/input/amazon-processed/X_processed_1754085057.625196.pt"
if os.path.exists(SAVED_VECTORIZED):
    X_processed = torch.load(SAVED_VECTORIZED, map_location="cpu")
    print("Loaded already-vectorized")
else:
    X_processed = preprocess(X)
    # memory
    print(X_processed.element_size() * X_processed.numel() / 1024**2, "MB")
    
    # cpu -> filesystem
    X_cpu = X_processed.coalesce().cpu()
    torch.save(X_cpu, f"/kaggle/working/X_processed_{time.time()}.pt")

Loaded already-vectorized
time: 3.76 s (started: 2025-08-12 01:26:41 +00:00)


In [12]:
print(len(bag_of_words))
print(len(X_processed))

LAYERS = 124, 42, 14, 6
# LAYER_1, LAYER_2 = 75, 5  # original
LAYER_1, LAYER_2, LAYER_3, LAYER_4 = LAYERS

bag_size = len(bag_of_words)

print("num params L1:", len(bag_of_words) * LAYER_1 + LAYER_1)
num_params = (
    sum(LAYERS[i - 1] * LAYERS[i] + LAYERS[i] for i in range(1, len(LAYERS)))
    + bag_size * LAYERS[0]
    + LAYERS[0]
    + LAYERS[-1]
    + 1
)
print("num params:", num_params)

3000
4880181
num params L1: 372124
num params: 378073
time: 969 µs (started: 2025-08-12 01:26:45 +00:00)


In [13]:
from torch.nn import init
import math

class SparseLinear(nn.Module):
    def __init__(self, in_features, out_features, bias=True, device=None, dtype=None):
        """Source copied from torch.nn.Linear @ https://github.com/pytorch/pytorch/blob/v2.7.0/torch/nn/modules/linear.py#L50"""
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(
            torch.empty((out_features, in_features), **factory_kwargs)
        )
        if bias:
            self.bias = nn.Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter("bias", None)
        self.reset_parameters()

    def forward(self, x):
        x_sparse = x.to(self.weight.dtype).coalesce()  # ensures unique indices
        out = torch.sparse.mm(
            x_sparse,  # (B x in) · (out x in)ᵀ -> (B x out)
            self.weight.t(),
        )
        if self.bias is not None:
            out = out + self.bias
        return out
    
    def reset_parameters(self) -> None:
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(in_features), 1/sqrt(in_features)). For details, see
        # https://github.com/pytorch/pytorch/issues/57109
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)


time: 1.37 ms (started: 2025-08-12 01:26:45 +00:00)


In [14]:
torch.serialization.add_safe_globals([nn.Sequential, nn.Linear, nn.Sigmoid, nn.ReLU])

time: 282 µs (started: 2025-08-12 01:26:45 +00:00)


In [15]:
if DO_VALIDATION:
    XT_VECTORIZED_PATH = "/kaggle/input/amazon-processed/Xt_processed_IndustrialScientific_1754088738.538544.pt"
    if os.path.exists(XT_VECTORIZED_PATH):
        X_test = torch.load(XT_VECTORIZED_PATH, map_location="cpu")
        print("Loaded saved vectorized validation")
    else:
        X_test = preprocess(Xt)
        Xt_cpu = X_test.coalesce().cpu()
        torch.save(Xt_cpu, f"/kaggle/working/Xt_processed_IndustrialScientific_{time.time()}.pt")

Loaded saved vectorized validation
time: 3.85 s (started: 2025-08-12 01:26:45 +00:00)


In [16]:
PRETRAIN_PATH = "/kaggle/input/amazon-processed/model_epoch6_1754644752.4333465.pth"


CONFIG = {
    "PARAMS": num_params,
    "BATCHES": 1000,
    "EPOCHS": 100,
    "SAMPLES": n_samples,#int(n_samples * 0.8),
    "LR": 0.003,
    "BAG_SIZE": bag_size,
}
if False:  # test runs
    CONFIG = {
        "PARAMS": num_params,
        "BATCHES": 1000,
        "EPOCHS": 10,
        "SAMPLES": int(n_samples * 0.8),
        "LR": 0.003,
        "BAG_SIZE": bag_size,
    }


print(CONFIG)

def build_model():
    # optimal params = 10% of 80000 = 8000
    model = nn.Sequential(
        SparseLinear(len(bag_of_words), LAYER_1),
        nn.ReLU(),
        nn.Linear(LAYER_1, LAYER_2),
        nn.ReLU(),
        nn.Linear(LAYER_2, LAYER_3),
        nn.ReLU(),
        nn.Linear(LAYER_3, LAYER_4),
        nn.ReLU(),
        nn.Linear(LAYER_4, 1),
        # nn.Sigmoid(),  # using logits loss
    ).to(DEVICE)  # 25 it/s -> 88 it/s
    model = torch.compile(model)
    # model = nn.DataParallel(model)  # use both T4s        

    # TODO: clear model if instantiating once
    return model


def sparse_collate_fn(batch):  # for batching
    inputs, targets = zip(*batch)
    # Stack sparse tensors along batch dimension
    batched_input = torch.stack(inputs, dim=0)
    batched_targets = torch.stack(targets, dim=0)
    return batched_input, batched_targets


def run_nn():
    # common data loading stuff
    
    n_samples = CONFIG["SAMPLES"]
    # X_train, X_test = X_processed[:n_samples], X_processed[n_samples:]
    # y_train, y_test = y[:n_samples].float(), y[n_samples:].float()

    X_train = X_processed.half().to(DEVICE)
    y_train = y.half().to(DEVICE)

    dataset = TensorDataset(X_train, y_train)
    dataloader = DataLoader(
        dataset, batch_size=len(X_train) // CONFIG["BATCHES"], shuffle=True,
        collate_fn=sparse_collate_fn

    )

    
    if not DO_VALIDATION:
        if os.path.exists(PRETRAIN_PATH):
            print(f"Loading pretrained model from {PRETRAIN_PATH}")
            model = torch.load(PRETRAIN_PATH, weights_only=False).to(DEVICE)
        else:
            print("No pretrained model found")
            model = build_fresh_model()

    
        print(model)
    
        loss_fn = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=CONFIG["LR"])
    
        # loss_per_batch = []
    
        start_time = time.time()

        # continue training
        for epoch in range(FINISHED_EPOCHS + 1, CONFIG["EPOCHS"]):
            print(f"[{time.time()}] Epoch {epoch}")
            for X_batch, y_batch in tqdm(dataloader):
                # already allocated, removing to(mps) shaved 33% of time
                X_batch, y_batch = X_batch.to(DEVICE, non_blocking=True), y_batch.to(DEVICE, non_blocking=True)
                optimizer.zero_grad()
                y_pred = model(X_batch)
                loss = loss_fn(y_pred.squeeze(-1), y_batch)
                loss.backward()
                optimizer.step()
                # loss_per_batch.append(loss.item())
    
            tm = time.time()
            torch.save(model, f"/kaggle/working/model_epoch{epoch}_{tm}.pth")
    
        end_time = time.time()
        print(f"Time taken: {end_time - start_time:.4f} seconds")
    
        return model
    
    else:
        global X_test
        X_test, y_test = X_test.half().to(DEVICE), yt.half().to(DEVICE)
        
        # load model from file
        model = torch.load(
            # "/kaggle/input/amazon-processed/model_epoch6_1754644752.4333465.pth", 
            "/kaggle/input/amazon-processed/model_epoch2_1754046175.4568558.pth",
            weights_only=False
        )

        with torch.no_grad():
            y_pred_train = model(X_train).squeeze(-1)
            y_pred_test = model(X_test).squeeze(-1)
    
        # sigmoid to binary
        train_preds = (y_pred_train >= 0).int()
        test_preds = (y_pred_test >= 0).int()


        print("positives in y_test :", torch.count_nonzero(y_test).item())      # no overflow
        print("positives predicted :", torch.count_nonzero(test_preds).item())
    
        num_true_train = (y_train == 1).sum()
        num_true_test = (y_test == 1).sum()
        train_acc = (train_preds == y_train.int()).float().mean()
        test_acc = (test_preds == y_test.int()).float().mean()
    
        # plt.xlabel("Batch")
        # plt.ylabel("Loss")
        # plt.plot(loss_per_batch)
        # plt.show()
        # print("Final loss:", loss_per_batch[-1])
    
        print(
            f"Train Accuracy: {train_acc:.4f} ({train_preds.sum()}/{len(X_train)}), random guess accuracy: {max(num_true_train, len(X_train) - num_true_train) / len(X_train):.4f}"
        )
        print(
            f"Test Accuracy: {test_acc:.4f} ({test_preds.sum()}/{len(X_test)}), random guess accuracy: {max(num_true_test, len(X_test) - num_true_test) / len(X_test):.4f}"
        )
    
        precision = (
            test_preds * y_test
        ).sum() / test_preds.sum()  # true pos / predicted pos
        recall = (test_preds * y_test).sum() / y_test.sum()  # true pos / actual pos
        f1 = 2 * precision * recall / (precision + recall)
        print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    
        return model


m = run_nn()
print(CONFIG)

{'PARAMS': 378073, 'BATCHES': 1000, 'EPOCHS': 100, 'SAMPLES': 4880181, 'LR': 0.003, 'BAG_SIZE': 3000}
positives in y_test : 3525718
positives predicted : 3746116
Train Accuracy: 0.8793 (3746116/4880181), random guess accuracy: 0.7225
Test Accuracy: 0.8793 (3746116/4880181), random guess accuracy: 0.7225
Precision: nan, Recall: nan, F1: nan
{'PARAMS': 378073, 'BATCHES': 1000, 'EPOCHS': 100, 'SAMPLES': 4880181, 'LR': 0.003, 'BAG_SIZE': 3000}
time: 9.83 s (started: 2025-08-12 01:26:49 +00:00)


```js
// 7749 vocab, 64 layer 1
Final loss: 0.43634021282196045
Train Accuracy: 0.8201 (61019/80000)
Test Accuracy: 0.8158 (15540/20000)

// 100 vocab, 104 layer 1, 20 epoch
Final loss: 0.48798397183418274
Train Accuracy: 0.7697 (62999/80000), random guess accuracy: 0.6713
Test Accuracy: 0.7691 (16053/20000), random guess accuracy: 0.6823
Precision: 0.7812, Recall: 0.9190, F1: 0.8445

// -> 100 batches
Final loss: 0.4811912477016449
Train Accuracy: 0.7857 (62716/80000), random guess accuracy: 0.6713
Test Accuracy: 0.7719 (15957/20000), random guess accuracy: 0.6823
Precision: 0.7846, Recall: 0.9176, F1: 0.8459

// -> 1000 batch SIZE
Final loss: 0.45420441031455994
Train Accuracy: 0.7831 (62935/80000), random guess accuracy: 0.6713
Test Accuracy: 0.7722 (15991/20000), random guess accuracy: 0.6823
Precision: 0.7842, Recall: 0.9190, F1: 0.8463

// -> 78 layer 1
Final loss: 0.4723219573497772
Train Accuracy: 0.7814 (62978/80000), random guess accuracy: 0.6713
Test Accuracy: 0.7721 (16049/20000), random guess accuracy: 0.6823
Precision: 0.7831, Recall: 0.9211, F1: 0.8465


// 75 layer 1, 5 layer 2
Final loss: 0.47183775901794434
Train Accuracy: 0.7830 (61135/80000), random guess accuracy: 0.6713
Test Accuracy: 0.7702 (15633/20000), random guess accuracy: 0.6823
Precision: 0.7894, Recall: 0.9044, F1: 0.8430

// epoch 2 full dataset
Train Accuracy: 0.8793 (3746116/4880181), random guess accuracy: 0.7225
Test Accuracy: 0.8793 (3746116/4880181), random guess accuracy: 0.7225

// epoch 6 full dataset
Train Accuracy: 0.8912 (3691016/4880181), random guess accuracy: 0.7225
Test Accuracy: 0.8912 (3691016/4880181), random guess accuracy: 0.7225

```

In [17]:
def param_to_word_mapping(model: nn.Module) -> dict[str, float]:
    # m is something like nn.Sequential(), which is a module
    params = list(model.parameters())

    mapping = zip(bag_of_words.keys(), params[0].flatten().tolist())

    return {k: v for k, v in sorted(mapping, key=lambda x: x[1], reverse=True)}


param_to_word_mapping(m)

{'intrusive': 0.5029951333999634,
 'seamlessly': 0.46484899520874023,
 'seamless': 0.45753806829452515,
 'love': 0.44659942388534546,
 'refreshing': 0.4431968629360199,
 'regret': 0.43712541460990906,
 'supports': 0.4329652786254883,
 'tvs': 0.4311315417289734,
 'awesome': 0.42098352313041687,
 'charm': 0.4147966504096985,
 '3': 0.4100094735622406,
 'usb': 0.3999478816986084,
 'flawless': 0.3832333981990814,
 'environment': 0.38274234533309937,
 'soooo': 0.3802756667137146,
 'wait': 0.3788659870624542,
 'excellent': 0.37378597259521484,
 'stop': 0.37000080943107605,
 'night': 0.3616675138473511,
 'loads': 0.35618194937705994,
 'wonderfully': 0.35443827509880066,
 'resources': 0.3543795645236969,
 'includes': 0.35032951831817627,
 'fantastic': 0.34911876916885376,
 'anytime': 0.34297579526901245,
 'luv': 0.342551052570343,
 'highly': 0.33842170238494873,
 'whats': 0.33405426144599915,
 '2018': 0.3335212469100952,
 'asap': 0.3313641846179962,
 'reviewing': 0.3310576379299164,
 'recommend

time: 50.8 ms (started: 2025-08-12 01:26:59 +00:00)


In [18]:
tm = time.time()
torch.save(m, f"/kaggle/working/model_{tm}.pth")
json.dump(bag_of_words, open(f"/kaggle/working/bag_of_words_{tm}.json", "w"))

time: 8.81 ms (started: 2025-08-12 01:26:59 +00:00)


In [19]:
# torch.serialization.add_safe_globals([nn.Sequential, nn.Linear, nn.Sigmoid, nn.ReLU])
# m2 = torch.load("model.pth")

time: 241 µs (started: 2025-08-12 01:26:59 +00:00)
